In [1]:
import pandas as pd

In [2]:
cpuc = pd.read_csv('data/cpuc_with_budget_and_resource.csv').iloc[:,1:]
cc = pd.read_csv('data/City_and_County_Boundaries.csv', usecols =['COUNTY', 'CITY'])

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,12,17,19,22,23,24,27,47,48,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
cc.head()

,COUNTY,CITY
0,San Bernardino County,Adelanto
1,Los Angeles County,Agoura Hills
2,Alameda County,Alameda
3,Alameda County,Albany
4,Los Angeles County,Alhambra


In [4]:
cc['CITY'] = cc.CITY.str.upper()

In [5]:
cc.head()

,COUNTY,CITY
0,San Bernardino County,ADELANTO
1,Los Angeles County,AGOURA HILLS
2,Alameda County,ALAMEDA
3,Alameda County,ALBANY
4,Los Angeles County,ALHAMBRA


In [6]:
cc.rename(columns={'COUNTY':'county', 'CITY':'city'}, inplace=True)

In [7]:
cc.head()

,county,city
0,San Bernardino County,ADELANTO
1,Los Angeles County,AGOURA HILLS
2,Alameda County,ALAMEDA
3,Alameda County,ALBANY
4,Los Angeles County,ALHAMBRA


In [8]:
cpuc.shape

(746373, 72)

In [9]:
df = cpuc.merge(cc, left_on='SiteCity', right_on='city', how='left')
df.shape

(746373, 74)

In [10]:
df[df.county.isna()].SiteCity.value_counts()

San Francisco       2043
VALENCIA            1998
Santa Rosa          1493
CITY OF INDUSTRY    1402
MC FARLAND          1265
                    ... 
RAYMOND                1
Paramount              1
Mill Valley            1
Calexico               1
twentynine             1
Name: SiteCity, Length: 1017, dtype: int64

In [11]:
nocounty = df[df.county.isna()].SiteCity.value_counts().index.to_list()

In [12]:
df.replace(to_replace='SAN LUIS OBISBO', value='SAN LUIS OBISPO', inplace=True)

In [13]:
df[df.county.isna()].SiteCity.value_counts()[df[df.county.isna()].SiteCity.value_counts()>=1000]

San Francisco       2043
VALENCIA            1998
Santa Rosa          1493
CITY OF INDUSTRY    1402
MC FARLAND          1265
PLUMAS LAKE         1249
SPRING VALLEY       1067
MARIPOSA            1034
SAN JUAN CAPO       1018
RAMONA              1003
WINCHESTER          1001
Name: SiteCity, dtype: int64

In [14]:
city_ag = pd.read_csv('data/city_ag_2.csv')
cities = pd.read_csv('data/cities_1.csv')

## instead of trying to map entire cpuc dataset, map only local government/k12 programs

Goal: put local government programs and k12 programs in one claims df with budget and resource

In [15]:
lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

In [16]:
k12 = pd.read_csv('data/cpuc_claims_k-12_programs_2017_2019.csv')

In [17]:
lg = cpuc[cpuc['Grouping'].isin(lglist)]

In [18]:
lg.SiteCity = lg.SiteCity.str.upper()

C:\Users\audre\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [19]:
br = cpuc[['Budget', 'Resource_Flag', 'ClaimID']]

In [20]:
k12_b = k12.merge(br, left_on='ClaimID', right_on='ClaimID', how='left')

In [21]:
lg[lg.SiteCity.str.startswith('B')].SiteCity.value_counts()

BAKERSFIELD      18569
BRENTWOOD          970
BURLINGAME         920
BERKELEY           409
BUELLTON           277
BENICIA            126
BLUE LAKE          121
BAKERSIFELD         75
BEVERLY HILLS       70
BUENA PARK          67
BAY POINT           65
BRIDGEPORT          58
BRISBANE            58
BAYSIDE             48
BOULDER CREEK       29
BLYTHE              26
BARSTOW             25
BALDWIN PARK        20
BUTTONWILLOW        19
BELMONT             13
BISHOP              12
BELVEDERE            7
BENTON               6
BALBOA               6
BRADLEY              5
BURNEY               5
BRIDGEVILLE          4
BODEGA BAY           3
BEN LOMOND           3
BOONVILLE            3
BETHEL ISLAND        2
BROWNS VALLEY        2
BREA                 2
BANTA                1
BROOKDALE            1
Name: SiteCity, dtype: int64

In [22]:
k12_b[k12_b.SiteCity.str.startswith('B')].SiteCity.value_counts()

BLOOMINGTON     61
BALDWIN PARK    52
BREA            42
BELLFLOWER      40
BUENA PARK      25
BURLINGAME      13
BERKELEY         9
BLYTHE           2
Name: SiteCity, dtype: int64

In [23]:
lg.replace(to_replace='BAKERSIFELD', value='BAKERSFIELD', inplace=True)

C:\Users\audre\anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [24]:
lg[lg.SiteCity.str.startswith('B')].SiteCity.value_counts()

BAKERSFIELD      18644
BRENTWOOD          970
BURLINGAME         920
BERKELEY           409
BUELLTON           277
BENICIA            126
BLUE LAKE          121
BEVERLY HILLS       70
BUENA PARK          67
BAY POINT           65
BRISBANE            58
BRIDGEPORT          58
BAYSIDE             48
BOULDER CREEK       29
BLYTHE              26
BARSTOW             25
BALDWIN PARK        20
BUTTONWILLOW        19
BELMONT             13
BISHOP              12
BELVEDERE            7
BENTON               6
BALBOA               6
BURNEY               5
BRADLEY              5
BRIDGEVILLE          4
BODEGA BAY           3
BOONVILLE            3
BEN LOMOND           3
BROWNS VALLEY        2
BREA                 2
BETHEL ISLAND        2
BANTA                1
BROOKDALE            1
Name: SiteCity, dtype: int64

In [25]:
k12_b.to_csv('data/k12_with_budget_and_resource.csv',index=False)

lg.to_csv('data/lg_with_budget_and_resource.csv', index=False)

_____

# Looking at percentage of budget for cities that have no demographic information

In [26]:
nc = cities[cities.city.isna()].SiteCity.to_list()

In [27]:
lg[lg.SiteCity.isin(nc)].SiteCity.value_counts()

CUTTEN         433
BUELLTON       277
GUADALUPE      270
SAN JOAQUIN    202
TIBURON        186
              ... 
SANTA NELLA      1
BANTA            1
VERNALIS         1
SUTTER           1
LOS MOLINOS      1
Name: SiteCity, Length: 119, dtype: int64

In [28]:
lg[lg.SiteCity.isin(nc)].Budget.sum()/lg.Budget.sum()*100

1.1664638581450064

1.1% of budget used occurs in these cities with no data, ok to chuck? weird because that's a quarter of the k12 data, and we decided 4.5% is significant enough to run a regression but 1.1% insignificant enough to chuck?? doesn't really add up if that is the reasoning..... might be able to manually reclassify some of the bigger places to bring this number down, but where to draw the line? ej: if goal is to bring budget percentage down to 0.5%, could either reclassify many little places or a few big places... not fair to whichever party is excluded from analysis... gotta ask team about this one, mostly concered about what we put in the final write up to justify the chucking mostly bc its compared with us saying 'oh 4.5% is good enough for regressions!' in the k12 place. so maybe will have to write something else like 'members of the energy industry wanted us to include k12 programs in our analysis'

In [29]:
k12_b[k12_b.SiteCity.isin(nc)].Budget.sum()/k12_b.Budget.sum()*100

0.005220801152839025

0.005% of no city budget for k12, def ok to chuck

_______

## **CITY COUNTY MAP???**

In [71]:
ccr = pd.read_csv('data/city_county_ruca.csv')

In [72]:
ccr.head()

,city,County,RUCA code
0,ACAMPO,SAN JOAQUIN,3
1,ACAMPO,SAN JOAQUIN,1
2,ACTON,LOS ANGELES,2
3,ADELANTO,SAN BERNARDINO,1
4,AGOURA HILLS,LOS ANGELES,2


In [73]:
cc = ccr[['city', 'County']]

In [76]:
cc.rename(columns={'County':'county'}, inplace=True)

C:\Users\audre\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [79]:
cc = cc.drop_duplicates()

dc = cc.city.value_counts()>1

double_cities = dc.reset_index()[dc.reset_index().city==True]['index'].to_list()

In [89]:
cc[cc.city.isin(double_cities)]

,city,county
4,AGOURA HILLS,LOS ANGELES
6,AGOURA HILLS,VENTURA
64,BERKELEY,CONTRA COSTA
65,BERKELEY,ALAMEDA
73,BISHOP,INYO
75,BISHOP,MONO
193,COTTONWOOD,TEHAMA
194,COTTONWOOD,SHASTA
228,DIXON,YOLO
229,DIXON,SOLANO


In [98]:
wrong_counties = [6,64,75,193,228,283,396,404,480,516,611,633,654,720,740,756,874,878,893,931,937,942]

In [99]:
cc.drop(index=wrong_counties, inplace=True)

In [94]:
cc.loc[515]=['MCARTHUR', 'SHASTA']

In [104]:
cc.reset_index(drop=True, inplace=True)

In [105]:
cc.head()

,city,county
0,ACAMPO,SAN JOAQUIN
1,ACTON,LOS ANGELES
2,ADELANTO,SAN BERNARDINO
3,AGOURA HILLS,LOS ANGELES
4,ALAMEDA,ALAMEDA


In [106]:
cc.to_csv('data/city_county_mapping.csv', index=False)